# Week 1 ex, part 2: API

According to Chapter 1, what are some things that we humans might want technology to help us get?

What's the maximum number of characters in a Tweet?

How many monthly active users does the book say that we can find on Twitter?

What is the key difference between connections on Twitter and connections on Facebook?

Take a look at the tweet meta-data on page 22-25. Which fields do you think are most interesting?


In [18]:
import twitter

## Example 1.1

In [19]:
CONSUMER_KEY = 'LjTdrUDxDx0fvfn86V22jIv7V'
CONSUMER_SECRET = 'ry83IMHNUvcNfgSlV5U6mOk0ZSUCdMxnyMwl1DigKgBdX3fxpR'
OAUTH_TOKEN = '770964598656761857-kCSU5iazwpwhqQ546ZIka7j1R6aUBUw'
OAUTH_TOKEN_SECRET = 'ZzCGn5kpwnH9xKfzTX0T9hrp7hfedK3ILflnj6ZhaGRyw'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(domain='api.twitter.com', 
                              api_version='1.1',
                              auth=auth
                             )

print twitter_api

## Example 1.2

In [20]:
# With an authenticated twitter_api in existence, you can now use it to query Twitter resources as usual.
# However, the trends resource is cleaned up a bit in v1.1, so requests are a bit simpler than in the latest
# printing. See https://dev.twitter.com/docs/api/1.1/get/trends/place

# The Yahoo! Where On Earth ID for the entire world is 1
WORLD_WOE_ID = 1 
DENMARK_ID = 23424796 

# Prefix id with the underscore for query string parameterization.
# Without the underscore, it's appended to the URL itself
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
denmark_trends = twitter_api.trends.place(_id=DENMARK_ID)
print denmark_trends


[{u'created_at': u'2016-09-06T08:15:05Z', u'trends': [{u'url': u'http://twitter.com/search?q=%23digitalfrontrunners', u'query': u'%23digitalfrontrunners', u'tweet_volume': None, u'name': u'#digitalfrontrunners', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%22Jacob+Nybroe%22', u'query': u'%22Jacob+Nybroe%22', u'tweet_volume': None, u'name': u'Jacob Nybroe', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%22Karina+Pedersen%22', u'query': u'%22Karina+Pedersen%22', u'tweet_volume': None, u'name': u'Karina Pedersen', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23EULCS', u'query': u'%23EULCS', u'tweet_volume': None, u'name': u'#EULCS', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23cphftw', u'query': u'%23cphftw', u'tweet_volume': None, u'name': u'#cphftw', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23dksocial', u'query': u'%23dksocial', u'tweet_volume': None, u'name': u'#dksoci

## Example 1.3

In [21]:
import json
print json.dumps(world_trends, indent=1)
print json.dumps(denmark_trends, indent=1)

[
 {
  "created_at": "2016-09-06T08:10:00Z", 
  "trends": [
   {
    "url": "http://twitter.com/search?q=%23FelizMartes", 
    "query": "%23FelizMartes", 
    "tweet_volume": null, 
    "name": "#FelizMartes", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23%D9%86%D8%B7%D8%A7%D9%84%D8%A8_%D8%A8%D8%AA%D8%AD%D8%B3%D9%8A%D9%86_%D8%A7%D9%84%D9%86%D8%AA_%D9%81%D9%8A_%D8%A7%D9%84%D8%B3%D8%B9%D9%88%D8%AF%D9%8A%D9%87", 
    "query": "%23%D9%86%D8%B7%D8%A7%D9%84%D8%A8_%D8%A8%D8%AA%D8%AD%D8%B3%D9%8A%D9%86_%D8%A7%D9%84%D9%86%D8%AA_%D9%81%D9%8A_%D8%A7%D9%84%D8%B3%D8%B9%D9%88%D8%AF%D9%8A%D9%87", 
    "tweet_volume": 13100, 
    "name": "#\u0646\u0637\u0627\u0644\u0628_\u0628\u062a\u062d\u0633\u064a\u0646_\u0627\u0644\u0646\u062a_\u0641\u064a_\u0627\u0644\u0633\u0639\u0648\u062f\u064a\u0647", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23%D8%A7%D8%AD%D9%84%D8%A7_%D9%85%D8%A7_%D9%81%D9%8A%D9%86%D9%8A", 
    "query": "%23%D8%A

## Example 1.4

In [22]:
world_trends_set = set([trend['name']
    for trend in world_trends[0]['trends']])

denmark_trends_set = set([trend['name']
    for trend in denmark_trends[0]['trends']])

common_trends = world_trends_set.intersection(denmark_trends_set)
print common_trends

set([])


## Example 1.5

In [34]:
# Like all other APIs, search requests now require authentication and have a slightly different request and
# response format. See https://dev.twitter.com/docs/api/1.1/get/search/tweets

q = "#NewYork"
count = 100

search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']

# v1.1 of Twitter's API provides a value in the response for the next batch of results that needs to be parsed out
# and passed back in as keyword args if you want to retrieve more than one page. It appears in the 'search_metadata'
# field of the response object and has the following form:'?max_id=313519052523986943&q=NCAA&include_entities=1'
# The tweets themselves are encoded in the 'statuses' field of the response


# Here's how you would grab five more batches of results and collect the statuses as a list
for _ in range(5): 
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError, e: # No more results when next_results doesn't exist
        break

    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ]) # Create a dictionary from the query string params
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

In [32]:
print len(statuses)
print json.dumps(statuses[0:2], indent=1) # Only print a couple of tweets here in IPython Notebook

97
[
 {
  "contributors": null, 
  "truncated": false, 
  "text": "Hello Old lady \ud83d\uddfd\ud83c\uddfa\ud83c\uddf8 #USA #NYC #statueofliberty #liberty #libertyisland #oldlady #newyork #n\u2026 https://t.co/fMHTRkjwXS https://t.co/J31VhzLf37", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 773075188015517696, 
  "favorite_count": 0, 
  "entities": {
   "symbols": [], 
   "user_mentions": [], 
   "hashtags": [
    {
     "indices": [
      19, 
      23
     ], 
     "text": "USA"
    }, 
    {
     "indices": [
      24, 
      28
     ], 
     "text": "NYC"
    }, 
    {
     "indices": [
      29, 
      45
     ], 
     "text": "statueofliberty"
    }, 
    {
     "indices": [
      46, 
      54
     ], 
     "text": "liberty"
    }, 
    {
     "indices": [
      55, 
      69
     ], 
     "text": "libertyisland"
    }, 
    {
     "indices": [
      70, 
      78
     ], 
     "text": "oldlady"
    }, 
    {
     "indices": [
      79, 
      87
     